# 内建函数

内置函数已预先声明。它们像任何其他函数一样被调用，但其中一些函数接受类型而不是表达式作为第一个参数。

内建函数：

    append cap close complex copy delete imag len
    make new panic print println real recover

内置函数没有标准的 Go 类型，因此它们只能调用而不能用作函数值。

## close

对于通道 c，内置函数 close(c) 标明不再在通道上发送值。如果是仅接收通道，则产生一个错误。发送到或关闭已关闭的通道会导致运行时死机。关闭 nil 通道也会导致运行时死机。调用 close 后，以及在收到任何先前发送的值后，接收操作将返回通道类型的零值，而不会阻塞。

## len 和 cap

函数 len 用于获得字符串所占字节数；数组、切片、映射 以及 通道缓存中排队 的元素的个数：

    string type      string length in bytes
    [n]T, *[n]T      array length (== n)
    []T              slice length
    map[K]T          map length (number of defined keys)
    chan T           number of elements queued in channel buffer
    
函数 cap 用于获得数组的元素个数；切片的容量以及通道缓存的容量：

    [n]T, *[n]T      array length (== n)
    []T              slice capacity
    chan T           channel buffer capacity
    
nil 切片、映射或通道的长度为 0。nil 切片和通道的容量为 0。

以下关系在任何时候都成立：

    0 <= len(s) <= cap(s)

若 s 为字符串常量，则表达式 len(s) 为常量。 若 s 的类型为数组或数组指针，且表达式 s 不包含 通道接收 或（非常量）函数调用，则表达式 len(s) 与 cap(s) 为常量，否则，不为常量：

In [20]:
const (
    c1 = imag(2i)                    // imag(2i) = 2.0 is a constant
    c2 = len([10]float64{2})         // [10]float64{2} contains no function calls
    c3 = len([10]float64{c1})        // [10]float64{c1} contains no function calls
    c4 = len([10]float64{imag(2i)})  // imag(2i) is a constant and no function call is issued
)
println(c1,c2,c3,c4)

2 10 10 10


In [19]:
var z complex128

const (
    c5 = len([10]float64{imag(z)})   // invalid: imag(z) is a (non-constant) function call
)

c5

10

## new

内建函数 new 接受类型 `T` 并返回类型为 `*T` 的值。其内存按照 **零值** 来初始化：

In [21]:
new(int)

0xc000422c28

In [22]:
*new(int)

0

In [27]:
new(struct { a int; b float32})

&{0 0}

## make

切片、映射与通道为引用类型，无需使用 new 来间接额外分配，而是用 make 来初始化。

make 接受类型 `T` 返回类型为 `T` 的值，也是按照 零值 来初始化。make 调用规则：

    调用             类型 T     结果

    make(T, n)       slice      类型为T，长度为n，容量为n的切片
    make(T, n, m)    slice      类型为T，长度为n，容量为m的切片

    make(T)          map        类型为T的映射
    make(T, n)       map        类型为T，初始空间为n个元素的映射

    make(T)          channel    类型为T的同步信道
    make(T, n)       channel    类型为T，缓存大小为n的异步信道

实参 n 和 m 必须为非负，且可表示为 int 类型的整数值，n 必不大于 m：

In [28]:
s := make([]int, 10, 100)       // len(s) == 10，cap(s) == 100 的切片
s := make([]int, 1e3)           // len(s) == cap(s) == 1000 的切片
c := make(chan int, 10)         // 缓存大小为10的信道
m := make(map[string]int, 100)  // 初始空间为100个元素的映射

In [29]:
s := make([]int, 1<<63)         // 非法：len(s) 不能表示为 int 类型的值
s := make([]int, 10, 0)         // 非法：len(s) > cap(s)

ERROR: untyped constant {int 9223372036854775808} overflows <int>

## append

内建函数 append 用于在切片中追加零个或多个元素，并返回生成的切片。追加的元素类型与切片中元素的类型要一致，作为一种特殊情况，`[]byte` 和 `[]rune` 类型切片，可以传 `字符串...`，自动追加字符串的字节和 Unicode 码值：

In [31]:
s0 := []int{0, 0}
s1 := append(s0, 2)
s1

[0 0 2]

In [32]:
s2 := append(s1, 3, 5, 7) 
s2

[0 0 2 3 5 7]

In [33]:
s3 := append(s2, s0...)
s4 := append(s3[3:6], s3[2:]...)

In [36]:
s3

[0 0 2 3 5 7 0 0]

In [37]:
s4

[3 5 7 2 3 5 7 0 0]

In [38]:
var t []interface{}
append(t, 42, 3.1415, "foo")

[42 3.1415 foo]

In [44]:
var b []byte
append(b, "bar"...)

[98 97 114]

In [47]:
var b []rune
append(b, '学','习')

[23398 20064]

In [48]:
var b []rune
append(b, "学习"...)

[23398 20064]

## copy

函数 copy 将元素从一个切片（来源 src）复制到另一个切片（目标 dst）。被复制的元素数量为 len(src) 与 len(dst) 中最小的那个。两个切片的类型要相同，作为一个特例，dst 的类型为 `[]byte` ，则 src 的类型可以是字符串：

In [8]:
var a = [...]int{0, 1, 2, 3, 4, 5, 6, 7}
var s = make([]int, 6)
copy(s, a[:])

In [9]:
s

[0 1 2 3 4 5]

In [10]:
copy(s, s[2:])
s

[2 3 4 5 4 5]

In [11]:
b := make([]byte, 5)
copy(b, "Hello, World!")

5

In [12]:
b

[72 101 108 108 111]

In [13]:
string(b)

Hello

In [19]:
r := make([]rune, 2)
copy(r, []rune("好好学习"))

In [20]:
r

[22909 22909]

## delete

内建函数 delete 从映射 m 中移除键为 k 的元素 `delete(m, k)`。若映射 m 为 nil 或元素不存在，则 delete 就是一个空操作。

In [21]:
m := map[string]int{"a":1, "b":2}
delete(m, "a")
m

map[b:2]

In [22]:
delete(m, "c")

In [27]:
delete(m, 98)

In [28]:
m

map[]

In [29]:
delete(m, 3.14) // 浮点数不能表示字符串

ERROR: cannot convert untyped constant {float64 157/50} to <string>

## complex，real 和 imag

内建函数 complex 从一个浮点数实部和虚部构造一个复数值。 而 real 和 imag 则提取一个复数值的实部和虚部。对于 complex，两实参必须为相同的浮点类型。

In [30]:
complex(2, -2)

(2-2i)

In [31]:
complex(float32(1), float32(1))

(1+1i)

In [32]:
complex(float32(1), float64(1))

ERROR: repl.go:1:29: mismatched types float32 and float64 in: complex(float32(1), float64(1))

In [33]:
real(1i)

0

In [34]:
imag(1i)

1

## panic 和 recover

内建函数 panic 和 recover 用于报告并处理运行时恐慌以及由程序定义的错误情况。

函数 F 中执行 panic 调用会终止 F 的执行，且不执行该函数剩余的部分。被 F *推迟* 函数都会在 F 返回给调用者前执行，直到程序被终止，而错误情况会被报告：

In [36]:
import "fmt"
func protect(g func()) {
    defer func() {
        fmt.Println("done")  // 即使有恐慌Println也会正常执行
    }()
    // panic("unreachable")
    fmt.Println("start")
    g()
}

func f() { fmt.Println("Hi")}

protect(f)

start
Hi
done


In [37]:
import "fmt"
func protect(g func()) {
    defer func() {
        fmt.Println("done")  // 即使有恐慌Println也会正常执行
    }()
    panic("unreachable")
    fmt.Println("start")
    g()
}

func f() { fmt.Println("Hi")}

protect(f)

done


ERROR: unreachable

recover 函数允许程序管理恐慌过程的行为。在 **推迟函数**中，执行 recover 调用会取回传递至 panic 调用的错误值来停止恐慌过程：

In [38]:
import "fmt"
func protect(g func()) {
    defer func() {
        fmt.Println("done")  // 即使有恐慌Println也会正常执行
        if x := recover(); x != nil {
            fmt.Printf("run time panic: %v", x)
        }
    }()
    // panic("unreachable")
    fmt.Println("start")
    g()
}

func f() { fmt.Println("Hi")}

protect(f)

start
Hi
done


In [39]:
import "fmt"
func protect(g func()) {
    defer func() {
        fmt.Println("done")  // 即使有恐慌Println也会正常执行
        if x := recover(); x != nil {
            fmt.Printf("run time panic: %v", x)
        }
    }()
    panic("unreachable")
    fmt.Println("start")
    g()
}

func f() { fmt.Println("Hi")}

protect(f)

done
run time panic: unreachable

## print 和 println

当前实现提供了几个在引导过程中有用的内建函数。这些函数因完整性而被保留， 但不保证会继续留在该语言中。它们并不返回结果。

    函数       行为

    print      打印所有实参；实参的格式取决于具体实现
    println    类似print，但会在实参之间打印空格并在末尾打印新行

In [40]:
print("a",1)

a1

In [41]:
println("a",1)

a 1
